In [14]:
import pandas as pd

In [15]:
sofr_raw = pd.read_excel("hist_data.xlsm", sheet_name="SofrCurve")
aapl_raw = pd.read_excel("hist_data.xlsm", sheet_name="AAPL").set_index("Date")
msft_raw = pd.read_excel("hist_data.xlsm", sheet_name="MSFT").set_index("Date")
f_raw = pd.read_excel("hist_data.xlsm", sheet_name="F").set_index("Date")
bac_raw = pd.read_excel("hist_data.xlsm", sheet_name="BAC").set_index("Date")

In [16]:
# clean up sofr raw
sofr_raw = sofr_raw.T
annualized_time = sofr_raw.loc["T"]
sofr_raw.drop(["T"], inplace=True)
sofr_raw.columns = sofr_raw.loc["Tenor"]
sofr_raw.drop(["Tenor"], inplace=True)
sofr_raw.columns.name = "Date"

In [17]:
sofr_raw

Date,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-10-31,0.039191,0.038721,0.03867,0.040536,0.044577,0.046004,0.046449,0.044583,0.042002,0.040318,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.03235,0.030552,0.028708
2022-11-01,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.04697,0.045022,0.042344,0.040614,...,0.036802,0.036682,0.036511,0.036287,0.03601,0.035678,0.033645,0.031979,0.030238,0.028478
2022-11-02,0.039948,0.039286,0.0391,0.040852,0.044884,0.04658,0.047203,0.045496,0.042749,0.040868,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
2022-11-03,0.040389,0.039585,0.03935,0.041154,0.045281,0.047107,0.047894,0.046594,0.043833,0.041825,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
2022-11-04,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.0475,0.046097,0.043385,0.041503,...,0.037687,0.037557,0.03738,0.037152,0.03687,0.036534,0.034424,0.032558,0.030723,0.028933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.053105,0.053084,0.053202,0.053424,0.05369,0.053293,0.052503,0.048399,0.045999,0.04485,...,0.044093,0.044056,0.043972,0.043841,0.043663,0.043437,0.041874,0.040238,0.038438,0.036493
2023-10-25,0.052981,0.053057,0.053259,0.05349,0.053749,0.053393,0.052653,0.048791,0.046595,0.045594,...,0.045289,0.045268,0.045198,0.045078,0.044906,0.044683,0.043076,0.041358,0.039564,0.037641
2023-10-26,0.053047,0.053089,0.053218,0.053414,0.053567,0.053089,0.052243,0.048044,0.045645,0.044538,...,0.044269,0.044244,0.044171,0.044049,0.043882,0.043668,0.042182,0.040589,0.03876,0.036764
2023-10-27,0.052989,0.05304,0.053185,0.053368,0.053486,0.052991,0.052115,0.047758,0.045284,0.0442,...,0.04446,0.044461,0.044411,0.044309,0.044157,0.043955,0.042508,0.040985,0.039167,0.037148


In [18]:
aapl_raw.rename(columns={"Adj Close": "adj_close_aapl"}, inplace=True)
msft_raw.rename(columns={"Adj Close": "adj_close_msft"}, inplace=True)
f_raw.rename(columns={"Adj Close": "adj_close_f"}, inplace=True)
bac_raw.rename(columns={"Adj Close": "adj_close_bac"}, inplace=True)

In [19]:
aapl_raw

,adj_close_aapl
Date,
2022-10-31,152.041122
2022-11-01,149.373917
2022-11-02,143.801514
2022-11-03,137.703613
2022-11-04,137.435455
...,...
2023-10-24,172.991058
2023-10-25,170.657135
2023-10-26,166.458023


## Moddeling the Joint Distribution of the Daily Risk Factor Changes ##
* For each risk factor, specify how you want to model their changes. As I suggested, one
way is to model the absolute daily changes in the zero rate risk factors and model the
daily relative returns of the stock prices.
* Once the above is specified, follow the approach of each VaR model (parametric/Monte
Carlo/historical) to model the joint distribution of the daily risk factor changes.

For each risk factor, you would need to use the 1-year historical data provided to compute their
daily changes over the 1-year historical period. Using the historical daily changes of the risk
factors, one can then determine the joint distribution of 1-day risk factor changes under each of
the 3 VaR models

* Parametric VaR. The 1-day risk factor changes are assumed to be multivariate-normally
distributed. One can simply estimate the means and covariance matrix of the multivariate
normal distribution using the sample means and convariance matrix calculated from the
historical daily changes.
* Monte Carlo VaR. The 1-day risk factor changes are also assumed to be
multivariate-normally distributed. Hence, one simply follow the same appproach as the
parametric VaR to estimate the means and covariance matrix of the multivariate normal
distribution.
* Historical VaR. The joint distribution of the 1-day risk factor changes is simply given by
the empirical distribution of the 1-year historical sample of daily risk factor changes.
Dr. Tony Wong 4 / 21


### Parametric VaR Data Prep ###

In [20]:
sofr_diff = sofr_raw.diff(1)
aapl_diff = aapl_raw.diff(1)
msft_diff = msft_raw.diff(1)
f_diff = f_raw.diff(1)
bac_diff = bac_raw.diff(1)

In [21]:
sofr_diff

Date,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-01,0.000413,0.000302,0.000216,0.000188,0.000272,0.000444,0.000521,0.000439,0.000342,0.000296,...,-0.000349,-0.000375,-0.000395,-0.000411,-0.000423,-0.000434,-0.000446,-0.000372,-0.000314,-0.00023
2022-11-02,0.000344,0.000262,0.000214,0.000128,0.000035,0.000132,0.000234,0.000474,0.000404,0.000254,...,0.000053,0.000019,-0.000013,-0.00004,-0.000058,-0.000064,-0.000018,-0.000043,0.000054,0.00013
2022-11-03,0.00044,0.000299,0.00025,0.000302,0.000397,0.000527,0.00069,0.001098,0.001085,0.000957,...,0.000366,0.000368,0.000388,0.000409,0.00042,0.000409,0.000184,0.000198,0.000115,0.000047
2022-11-04,0.005576,0.002758,-0.000555,-0.000543,-0.000069,-0.000355,-0.000394,-0.000497,-0.000448,-0.000322,...,0.000466,0.000488,0.000494,0.000495,0.000498,0.000512,0.000613,0.000424,0.000315,0.000278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.000015,0.000007,0.000011,0.000019,0.000052,0.000083,0.000114,0.000371,0.000417,0.000343,...,-0.000161,-0.000174,-0.000183,-0.000189,-0.000196,-0.000203,-0.000255,-0.000298,-0.000283,-0.000242
2023-10-25,-0.000124,-0.000027,0.000057,0.000066,0.000059,0.0001,0.00015,0.000393,0.000596,0.000743,...,0.001196,0.001212,0.001226,0.001237,0.001244,0.001246,0.001202,0.001119,0.001126,0.001147
2023-10-26,0.000066,0.000032,-0.000041,-0.000076,-0.000182,-0.000304,-0.00041,-0.000747,-0.00095,-0.001055,...,-0.00102,-0.001024,-0.001027,-0.001028,-0.001025,-0.001015,-0.000895,-0.000769,-0.000804,-0.000877
2023-10-27,-0.000059,-0.000049,-0.000033,-0.000046,-0.000082,-0.000099,-0.000129,-0.000286,-0.000361,-0.000339,...,0.000191,0.000217,0.00024,0.000259,0.000275,0.000287,0.000327,0.000396,0.000407,0.000384


In [22]:
msft_diff

,adj_close_msft
Date,
2022-10-31,NaN
2022-11-01,-3.914170
2022-11-02,-7.976593
2022-11-03,-5.782287
2022-11-04,7.057358
...,...
2023-10-24,1.207550
2023-10-25,10.119476
2023-10-26,-12.754119


In [23]:
# combine df?
combined_diff = pd.concat([sofr_diff, aapl_diff, msft_diff, f_diff, bac_diff], axis=1)
combined_diff.dropna(inplace=True)

In [24]:
combined_diff

,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,19Y,20Y,25Y,30Y,35Y,40Y,adj_close_aapl,adj_close_msft,adj_close_f,adj_close_bac
2022-11-01,0.000413,0.000302,0.000216,0.000188,0.000272,0.000444,0.000521,0.000439,0.000342,0.000296,...,-0.000423,-0.000434,-0.000446,-0.000372,-0.000314,-0.00023,-2.667205,-3.914170,0.026868,0.154266
2022-11-02,0.000344,0.000262,0.000214,0.000128,0.000035,0.000132,0.000234,0.000474,0.000404,0.000254,...,-0.000058,-0.000064,-0.000018,-0.000043,0.000054,0.00013,-5.572403,-7.976593,-0.304508,-0.106064
2022-11-03,0.00044,0.000299,0.00025,0.000302,0.000397,0.000527,0.00069,0.001098,0.001085,0.000957,...,0.00042,0.000409,0.000184,0.000198,0.000115,0.000047,-6.097901,-5.782287,0.179123,-0.192829
2022-11-04,0.005576,0.002758,-0.000555,-0.000543,-0.000069,-0.000355,-0.000394,-0.000497,-0.000448,-0.000322,...,0.000498,0.000512,0.000613,0.000424,0.000315,0.000278,-0.268158,7.057358,0.223904,0.867749
2022-11-07,-0.005741,-0.003085,0.000665,0.000813,0.000243,0.000437,0.000429,0.000536,0.000499,0.000413,...,0.000512,0.000514,0.000498,0.000577,0.000577,0.000562,0.536286,6.405014,0.170166,0.212116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.000015,0.000007,0.000011,0.000019,0.000052,0.000083,0.000114,0.000371,0.000417,0.000343,...,-0.000196,-0.000203,-0.000255,-0.000298,-0.000283,-0.000242,0.438858,1.207550,-0.088617,-0.099209
2023-10-25,-0.000124,-0.000027,0.000057,0.000066,0.000059,0.0001,0.00015,0.000393,0.000596,0.000743,...,0.001244,0.001246,0.001202,0.001119,0.001126,0.001147,-2.333923,10.119476,0.147696,0.079367
2023-10-26,0.000066,0.000032,-0.000041,-0.000076,-0.000182,-0.000304,-0.00041,-0.000747,-0.00095,-0.001055,...,-0.001025,-0.001015,-0.000895,-0.000769,-0.000804,-0.000877,-4.199112,-12.754119,-0.187082,0.565486
2023-10-27,-0.000059,-0.000049,-0.000033,-0.000046,-0.000082,-0.000099,-0.000129,-0.000286,-0.000361,-0.000339,...,0.000275,0.000287,0.000327,0.000396,0.000407,0.000384,1.326553,1.916107,-1.368659,-0.942478


In [25]:
combined_diff.mean()

1D                0.000053
1M                0.000058
2M                0.000064
3M                0.000057
6M                0.000035
9M                0.000025
1Y                0.000018
2Y                0.000007
3Y                0.000007
4Y                0.000012
5Y                0.000015
6Y                0.000017
7Y                0.000020
8Y                0.000021
9Y                0.000023
10Y               0.000024
11Y               0.000025
12Y               0.000026
13Y               0.000026
14Y               0.000027
15Y               0.000027
16Y               0.000027
17Y               0.000028
18Y               0.000028
19Y               0.000029
20Y               0.000029
25Y               0.000032
30Y               0.000033
35Y               0.000033
40Y               0.000032
adj_close_aapl    0.054441
adj_close_msft    0.405430
adj_close_f      -0.010927
adj_close_bac    -0.039395
dtype: float64

In [26]:
combined_diff.cov()

C:\Users\kilo\AppData\Local\Temp\ipykernel_16344\961700286.py:1: FutureWarning: The default value of numeric_only in DataFrame.cov is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  combined_diff.cov()


,adj_close_aapl,adj_close_msft,adj_close_f,adj_close_bac
adj_close_aapl,6.053548,7.445720,0.225533,0.360476
adj_close_msft,7.445720,24.802510,0.402358,0.587850
adj_close_f,0.225533,0.402358,0.072870,0.059933
adj_close_bac,0.360476,0.587850,0.059933,0.242192
